# Activity 12.01 - Understanding power usage

In this activity, as a data analyst involved in an energy conservation study, you are provided with data collected from a home in France over several years, which includes frequent measurements of total power usage as well as submeters that isolate the kitchen, laundry room, and some heating and air conditioning use. You have been asked to look at kitchen energy use and understand the time of day trends.

1. For this activity, all you will need is the pandas and numpy libraries. Load them in the first cell of the notebook:

In [1]:
import pandas as pd
import numpy as np

2. Read in the household_power_consumption.csv data from the Datasets directory, and list the first few rows.

3. You should inspect the data types of the columns, and the further investigate whether there are non-numeric values. If so, correct them by converting to NA valeus and then filling them by interpolation.

4. Make a quick visualization to understand the timeframe of the data. Your plan is to identify a year with complete data and focus on that:

5. Using the your you identified, create a new DataFrame having Date, Time and Kitchen_power_use. Not that Sub_metering_1 is the kitchen:

6. Date and Time are substrings; combine them on each row, then convert the combined string to a datetime, and store that in a new column called timestamp. Keep in mind the European format of the original date strings.

7. Create an hour and date column using methods on the timestamp column to represent the hour of the day and the date in a standard format:

8. Group the data by date and hour aggregating Kitchen_power_use:

9. For January, aggregate the data by hour and make a bar plot by hour of kitchen energy use:

10. You see that usage seems to begin around breakfast time, continue through the day with a peak at dinner time, then trail off. Make a similar plot for the entire year to compare: